## Download of company published financial statements using LSEG API

The Fundamental and Reference module provides the access to private and public company information via "TR."

- Statements include:
    - Balance sheets
    - Cash Flow
    - Income Statements
    
---
### Balance Sheets

In [1]:
import lseg.data as ld
import pandas as pd

In [2]:
# Read in company names to look up buy/sell recommendations for
sp400_companies = pd.read_csv("data/sp400_companies.csv", dtype={"CIK": str})
sp500_companies = pd.read_csv("data/sp500_companies.csv", dtype={"CIK": str})
sp600_companies = pd.read_csv("data/sp600_companies.csv", dtype={"CIK": str})

# Combine all CIKs into a single list
sp400_ciks = sp400_companies["CIK"].tolist()
sp500_ciks = sp500_companies["CIK"].tolist()
sp600_ciks = sp600_companies["CIK"].tolist()
ciks = sp400_ciks + sp500_ciks + sp600_ciks

In [3]:
# Start a session with the LSEG Data Platform (needs Refinitiv Workspace to be running in the background)
ld.open_session()

An error occurred while requesting URL('http://localhost:9010/api/status').
	ConnectError('[WinError 10061] Es konnte keine Verbindung hergestellt werden, da der Zielcomputer die Verbindung verweigerte')


: 

: 

- First we download the Refinitiv Instrument Code (RIC) for every company, because apparently downloading balance sheets does not work with CIKs

In [ ]:
rics = ld.get_data(
    universe = ciks,
    fields=[
        "TR.RIC"
    ],
)

c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_access_layer\_containers.py:62:UserWarning: You have duplicated instruments in your input. Output will contain unique instruments only.


In [ ]:
rics.head()

,Instrument,RIC
0,0001675149,AA.N
1,0000006201,AAL.OQ
2,0000824142,AAON.OQ
3,0001520697,ACHC.OQ
4,0001646972,ACI.N


In [2]:
# Save rics to a CSV file
#rics.to_csv("data/rics.csv", index=False)
rics = pd.read_csv("data/rics.csv", dtype={"Instrument": str})
rics.head()

,Instrument,RIC
0,0001675149,AA.N
1,0000006201,AAL.OQ
2,0000824142,AAON.OQ
3,0001520697,ACHC.OQ
4,0001646972,ACI.N


In [3]:
# Convert RICs to a list
ric_list = rics["RIC"].tolist()

- Download of balance sheets

Unfortunately, when not specifying a certain report period or type e.g. 10-K or 10-Q, only data that had been submitted in yearly reports (10-Ks) is downloaded.
Therefore, for every given date/year (?) we have to specifically download the balance sheets from:
- The last fiscal quarter (FQ0)
- The previous fiscal quarter (FQ-1)
- Fiscal quarter minus two (FQ-2)
- Fiscal quarter minus 3 (FQ-3)

The same logic is applied later on when downloading cash flow and income statements.

In [98]:
# Initialize an empty list to store DataFrames
results = []
# Loop through each RIC and download the balance sheet data
for ric in ric_list: 
    # Download income statements for each RIC and every quarter separately
    try:
        df1 = ld.get_history(
            universe=[ric],
            start='2000-01-01',
            end='2025-01-01',
            fields=[
                "TR.F.BalanceSheet(Period = FQ0)",
                "TR.F.BalanceSheet(Period=FQ0).FccItemName",
            ],
        )
        # Add RIC column to the DataFrame to identify the company
        df1['RIC'] = ric
        df2 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.BalanceSheet(Period = FQ-1)",
                "TR.F.BalanceSheet(Period=FQ-1).FccItemName",
            ],
        )
        df2['RIC'] = ric
        df3 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.BalanceSheet(Period = FQ-2)",
                "TR.F.BalanceSheet(Period=FQ-2).FccItemName",
            ],
        )
        df3['RIC'] = ric
        df4 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.BalanceSheet(Period = FQ-3)",
                "TR.F.BalanceSheet(Period=FQ-3).FccItemName",
            ],
        )   
        df4['RIC'] = ric
    except:
        # If an error occurs, current RIC is skipped
        continue
    # Concatenate the DataFrames for each RIC
    dfs = pd.concat([df1, df2, df3, df4])
    # Append the concatenated DataFrame to the results list
    results.append(dfs)

# Concatenate all DataFrames in the results list into a single DataFrame
balance_sheets = pd.concat(results)

c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `

In [99]:
# Work with a copy just to be safe
df = balance_sheets.copy()

# Reset index and rename to Date
df = df.reset_index().rename(columns={'index': 'Date'})

# Indicate statement type
df["Statement"] = "balance_sheet"

# Get rid of timestamp
df["Date"] = pd.to_datetime(df["Date"]).dt.date
df

,Date,STD Balance Sheet All,FCC Item Name,RIC,Statement
0,2011-09-30,<NA>,,AA.N,balance_sheet
1,2012-09-30,<NA>,,AA.N,balance_sheet
2,2013-09-30,<NA>,,AA.N,balance_sheet
3,2014-09-30,<NA>,,AA.N,balance_sheet
4,2016-06-30,332000000.0,TR.F.CashSTInvst,AA.N,balance_sheet
...,...,...,...,...,...
12237114,2023-03-31,556600000.0,TR.F.TotDebtExclIslamic,ZWS.N,balance_sheet
12237115,2023-03-31,223000000.0,TR.F.TradeAcctTradeNotesRcvblNetTot,ZWS.N,balance_sheet
12237116,2023-03-31,223300000.0,TR.F.CurrLiabExclCurrDebtTot,ZWS.N,balance_sheet
12237117,2023-03-31,607000000.0,TR.F.CurrAssetsExclCashSTInvstTot,ZWS.N,balance_sheet


In [100]:
# Save the data to a CSV file
df.to_csv("data/balance_sheets.csv", index=False)

In [7]:
ld.close_session() 

---
### Cash flow statements

In [2]:
rics = pd.read_csv("data/rics.csv", dtype={"RIC": str})
ric_list = rics["RIC"].tolist()

In [3]:
ld.open_session()

<lseg.data.session.Definition object at 0x1674e8f4950 {name='workspace'}>

In [ ]:
rics = ld.get_data(
    universe = ciks,
    fields=[
        "TR.RIC"
    ],
)

c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_access_layer\_containers.py:62:UserWarning: You have duplicated instruments in your input. Output will contain unique instruments only.


In [5]:
# Save RIC to CSV
rics.to_csv("data/rics.csv", index=False)

In [6]:
# Convert RICs to a list
ric_list = rics["RIC"].tolist()

In [ ]:
# Initialize an empty list to store DataFrames
results = []

# Loop through each RIC and download the balance sheet data
for i, ric in enumerate(ric_list):
    print(f"Processing RIC {i+1}/{len(ric_list)}: {ric}")
    # Download income statements for each RIC and every quarter separately
    try:
        df1 = ld.get_history(
            universe=[ric],
            start='2000-01-01',
            end='2025-01-01',
            fields=[
                "TR.F.CashFlowStatement(Period = FQ0)",
                "TR.F.CashFlowStatement(Period=FQ0).FccItemName",
            ],
        )
        # Add RIC column to the DataFrame to identify the company
        df2 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.CashFlowStatement(Period = FQ-1)",
                "TR.F.CashFlowStatement(Period=FQ-1).FccItemName",
            ],
        )
        df3 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.CashFlowStatement(Period = FQ-2)",
                "TR.F.CashFlowStatement(Period=FQ-2).FccItemName",
            ],
        )
        df4 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.CashFlowStatement(Period = FQ-3)",
                "TR.F.CashFlowStatement(Period=FQ-3).FccItemName",
            ],
        )   
    except:
        continue
    # Concatenate the DataFrames for each RIC
    dfs = pd.concat([df1, df2, df3, df4])
    dfs["RIC"] = ric  # Add RIC column to the concatenated DataFrame

    # Check if data was availablce, since balance sheet df lacks ~150 companies
    if dfs.empty:
        print(f"No data found for RIC {ric}.")
        continue

    # Append the concatenated DataFrame to the results list
    results.append(dfs)

# Concatenate all DataFrames in the results list into a single DataFrame
cash_flow_statements = pd.concat(results)

In [109]:
df = cash_flow_statements.copy()
# Reset index and rename to Date
df = df.reset_index().rename(columns={'index': 'Date'})

# Get rid of timestamp
df["Date"] = pd.to_datetime(df["Date"]).dt.date

# Create column to indicate type of statement
df["statement"] = "cashflow"
df

,Date,STD Cash Flow All,FCC Item Name,RIC,statement
0,2011-09-30,<NA>,,AA.N,cashflow
1,2012-09-30,<NA>,,AA.N,cashflow
2,2013-09-30,<NA>,,AA.N,cashflow
3,2014-09-30,<NA>,,AA.N,cashflow
4,2016-06-30,-19000000.0,TR.F.ProfLossStartingLineCF,AA.N,cashflow
...,...,...,...,...,...
4883991,2023-03-31,81000000.0,TR.F.CashDivPaidComStockBuybackNet,LNC.N,cashflow
4883992,2023-03-31,4000000.0,TR.F.ComStockBuybackNet,LNC.N,cashflow
4883993,2023-03-31,-774000000.0,TR.F.FreeCashFlowToEq,LNC.N,cashflow
4883994,2023-03-31,-876000000.0,TR.F.FOCF,LNC.N,cashflow


In [110]:
# Save to CSV
df.to_csv("data/cash_flow_statements.csv", index=False)

In [111]:
ld.close_session()

---
### Income statements

In [5]:
ld.open_session()

<lseg.data.session.Definition object at 0x22d72f11650 {name='workspace'}>

In [ ]:
# Initialize an empty list to store DataFrames
results = []
empty_count = 0

# Loop through each RIC and download the balance sheet data
for i, ric in enumerate(ric_list):
    print(f"Processing RIC {i+1}/{len(ric_list)}: {ric}")
    # Download income statements for each RIC and every quarter separately
    try:
        df1 = ld.get_history(
            universe=[ric],
            start='2000-01-01',
            end='2025-01-01',
            fields=[
                "TR.F.IncomeStatement(Period = FQ0)",
                "TR.F.IncomeStatement(Period=FQ0).FccItemName",
            ],
        )
        # Add RIC column to the DataFrame to identify the company
        df2 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.IncomeStatement(Period = FQ-1)",
                "TR.F.IncomeStatement(Period=FQ-1).FccItemName",
            ],
        )
        df3 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.IncomeStatement(Period = FQ-2)",
                "TR.F.IncomeStatement(Period=FQ-2).FccItemName",
            ],
        )
        df4 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.IncomeStatement(Period = FQ-3)",
                "TR.F.IncomeStatement(Period=FQ-3).FccItemName",
            ],
        )   
    except:
        continue
    # Concatenate the DataFrames for each RIC
    dfs = pd.concat([df1, df2, df3, df4])
   
    dfs["RIC"] = ric 
    
    if dfs.empty:
        print(f"No data found for RIC {ric}.")
        empty_count += 1
        continue 
    
    # Append the concatenated DataFrame to the results list
    results.append(dfs)

# Concatenate all DataFrames in the results list into a single DataFrame
income_statements = pd.concat(results)

print(f"Number of RICs with no data: {empty_count}")

In [7]:
df = income_statements.copy()

# Reset index and rename to Date
df = df.reset_index().rename(columns={'index': 'Date'})

# Get rid of timestamp
df["Date"] = pd.to_datetime(df["Date"]).dt.date

# Create column to indicate type of statement
df["statement"] = "income_statement"
df

,Date,STD Income Statement All,FCC Item Name,RIC,statement
0,2011-09-30,<NA>,,AA.N,income_statement
1,2012-09-30,<NA>,,AA.N,income_statement
2,2013-09-30,<NA>,,AA.N,income_statement
3,2014-09-30,<NA>,,AA.N,income_statement
4,2016-06-30,2323000000.0,TR.F.RevGoodsSrvc,AA.N,income_statement
...,...,...,...,...,...
10868820,2023-03-31,29644164.03785,TR.F.TaxAdjOpInc,ZWS.N,income_statement
10868821,2023-03-31,303700000.0,TR.F.OpExpnExclNonCashChrgTot,ZWS.N,income_statement
10868822,2023-03-31,45600000.0,TR.F.IncAvailToComShrBefDeprAmort,ZWS.N,income_statement
10868823,2023-03-31,9600000.0,TR.F.FixedChrg,ZWS.N,income_statement


In [8]:
# Save the data to a CSV file
df.to_csv("data/income_statements.csv", index=False)

In [9]:
ld.close_session()

---

### Inspecting the data and downloading missing values

In [4]:
balance_sheets = pd.read_csv("data/balance_sheets.csv", dtype={"RIC": str})
income_statements = pd.read_csv("data/income_statements.csv", dtype={"RIC": str})
cash_flow_statements = pd.read_csv("data/cash_flow_statements.csv", dtype={"RIC": str})

In [5]:
balance_sheets["RIC"].nunique(), income_statements["RIC"].nunique(), cash_flow_statements["RIC"].nunique()

(1338, 1453, 1203)

We started with 1505 unique CIKs, for which 1499 unique RICs could be looked up. However, in some of the downloaded financial statements, almost 300 RICs appear to be missing.  
Therefore, after determining which RICs are missing, I will try to download the left out data in the cells below.

In [ ]:
# Check which RICs are missing in each DataFrame
missing_balance_sheets = set(ric_list) - set(balance_sheets["RIC"].unique())
missing_cash_flow_statements = set(ric_list) - set(cash_flow_statements["RIC"].unique())
missing_income_statements = set(ric_list) - set(income_statements["RIC"].unique())

In [19]:
# Convert to list for download
missing_balance_sheets_list = list(missing_balance_sheets)
missing_cash_flow_statements_list = list(missing_cash_flow_statements)
missing_income_statements_list = list(missing_income_statements)

In [17]:
ld.open_session()

<lseg.data.session.Definition object at 0x2a704318990 {name='workspace'}>

- Try downloading missing balance sheets once more

In [20]:
# Initialize an empty list to store DataFrames
results = []
# Loop through each RIC and download the balance sheet data
for i, ric in enumerate(missing_balance_sheets_list):
    print(f"Processing RIC {i+1}/{len(missing_balance_sheets_list)}: {ric}") 
    # Download income statements for each RIC and every quarter separately
    try:
        df1 = ld.get_history(
            universe=[ric],
            start='2000-01-01',
            end='2025-01-01',
            fields=[
                "TR.F.BalanceSheet(Period = FQ0)",
                "TR.F.BalanceSheet(Period=FQ0).FccItemName",
            ],
        )
        # Add RIC column to the DataFrame to identify the company
        df1['RIC'] = ric
        df2 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.BalanceSheet(Period = FQ-1)",
                "TR.F.BalanceSheet(Period=FQ-1).FccItemName",
            ],
        )
        df2['RIC'] = ric
        df3 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.BalanceSheet(Period = FQ-2)",
                "TR.F.BalanceSheet(Period=FQ-2).FccItemName",
            ],
        )
        df3['RIC'] = ric
        df4 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.BalanceSheet(Period = FQ-3)",
                "TR.F.BalanceSheet(Period=FQ-3).FccItemName",
            ],
        )   
        df4['RIC'] = ric
    except:
        # If an error occurs, current RIC is skipped
        continue
    # Concatenate the DataFrames for each RIC
    
    # Check for failed downloads
    dfs = pd.concat([df1, df2, df3, df4])
    if df1.empty:
        print(f"df1 empty for RIC {ric}.")
    elif df2.empty:
        print(f"df2 empty for RIC {ric}.")
    elif df3.empty:
        print(f"df3 empty for RIC {ric}.")
    elif df4.empty:
        print(f"df4 empty for RIC {ric}.")  
        continue

    # Append the concatenated DataFrame to the results list
    results.append(dfs)

# Concatenate all DataFrames in the results list into a single DataFrame
balance_sheets = pd.concat(results)

Processing missing RIC 1/156: STEP.OQ


c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `

Processing missing RIC 2/156: UCB.N


c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `

Processing missing RIC 3/156: NHC.A
Processing missing RIC 4/156: MTUS.N


...

c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Processing missing RIC 154/156: TDC.N


c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `

Processing missing RIC 155/156: HTO.OQ
Processing missing RIC 156/156: VYX.N


C:\Users\benny\AppData\Local\Temp\ipykernel_25512\3093137973.py:68:FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
C:\Users\benny\AppData\Local\Temp\ipykernel_25512\3093137973.py:68:FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


Manually trying to download the now remaining data, i.e. df1 for GEV.N, RAL.N and MRP.N, as well as df3 for TTGT.OQ returned empty dataframes. Therefore, these reports have to be left out unfortunately.

In [23]:
# Work with a copy just to be safe
df = balance_sheets.copy()

# Reset index and rename to Date
df = df.reset_index().rename(columns={'index': 'Date'})

# Indicate statement type
df["Statement"] = "balance_sheet"

# Get rid of timestamp
df["Date"] = pd.to_datetime(df["Date"]).dt.date

# Save to CSV
df.to_csv("data/missing_balance_sheets.csv", index=False)

- Try downloading missing cash flow statements

In [25]:
# Initialize an empty list to store DataFrames
results = []

# Loop through each RIC and download the balance sheet data
for i, ric in enumerate(missing_cash_flow_statements_list):
    print(f"Processing RIC {i+1}/{len(missing_cash_flow_statements_list)}: {ric}")
    # Download income statements for each RIC and every quarter separately
    try:
        df1 = ld.get_history(
            universe=[ric],
            start='2000-01-01',
            end='2025-01-01',
            fields=[
                "TR.F.CashFlowStatement(Period = FQ0)",
                "TR.F.CashFlowStatement(Period=FQ0).FccItemName",
            ],
        )
        # Add RIC column to the DataFrame to identify the company
        df2 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.CashFlowStatement(Period = FQ-1)",
                "TR.F.CashFlowStatement(Period=FQ-1).FccItemName",
            ],
        )
        df3 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.CashFlowStatement(Period = FQ-2)",
                "TR.F.CashFlowStatement(Period=FQ-2).FccItemName",
            ],
        )
        df4 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.CashFlowStatement(Period = FQ-3)",
                "TR.F.CashFlowStatement(Period=FQ-3).FccItemName",
            ],
        )   
    except:
        continue
    # Concatenate the DataFrames for each RIC
    dfs = pd.concat([df1, df2, df3, df4])
    
    # Chech for failed downloads
    if df1.empty:
        print(f"df1 empty for RIC {ric}.")
    elif df2.empty:
        print(f"df2 empty for RIC {ric}.")
    elif df3.empty:
        print(f"df3 empty for RIC {ric}.")
    elif df4.empty:
        print(f"df4 empty for RIC {ric}.")  
        continue
    dfs["RIC"] = ric  # Add RIC column to the concatenated DataFrame

    # Check if data was availablce, since balance sheet df lacks ~150 companies
    if dfs.empty:
        print(f"No data found for RIC {ric}.")
        continue

    # Append the concatenated DataFrame to the results list
    results.append(dfs)

# Concatenate all DataFrames in the results list into a single DataFrame
cash_flow_statements = pd.concat(results)

Processing RIC 1/291: MCRI.OQ
Processing RIC 2/291: STEP.OQ


c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `

Processing RIC 3/291: REX.N
Processing RIC 4/291: UCB.N


c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `

Processing RIC 5/291: MODG.N
Processing RIC 6/291: ROCK.OQ
Processing RIC 7/291: PRSU.N
Processing RIC 8/291: MOGa.N
Processing RIC 9/291: NHC.A
Processing RIC 10/291: MTUS.N


...

Processing RIC 285/291: SLG.N
Processing RIC 286/291: STC.N
Processing RIC 287/291: SHOO.OQ
Processing RIC 288/291: SXT.N


c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Processing RIC 289/291: TDC.N


c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `

Processing RIC 290/291: HTO.OQ
Processing RIC 291/291: SCSC.OQ


In [26]:
# Work with a copy just to be safe
df = cash_flow_statements.copy()

# Reset index and rename to Date
df = df.reset_index().rename(columns={'index': 'Date'})

# Indicate statement type
df["Statement"] = "cash_flow_statement"

# Get rid of timestamp
df["Date"] = pd.to_datetime(df["Date"]).dt.date

# Save to CSV
df.to_csv("data/missing_cash_flow_statements.csv", index=False)

- Try downloading missing income statements

In [27]:
# Initialize an empty list to store DataFrames
results = []
empty_count = 0

# Loop through each RIC and download the balance sheet data
for i, ric in enumerate(missing_income_statements_list):
    print(f"Processing RIC {i+1}/{len(missing_income_statements_list)}: {ric}")
    # Download income statements for each RIC and every quarter separately
    try:
        df1 = ld.get_history(
            universe=[ric],
            start='2000-01-01',
            end='2025-01-01',
            fields=[
                "TR.F.IncomeStatement(Period = FQ0)",
                "TR.F.IncomeStatement(Period=FQ0).FccItemName",
            ],
        )
        # Add RIC column to the DataFrame to identify the company
        df2 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.IncomeStatement(Period = FQ-1)",
                "TR.F.IncomeStatement(Period=FQ-1).FccItemName",
            ],
        )
        df3 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.IncomeStatement(Period = FQ-2)",
                "TR.F.IncomeStatement(Period=FQ-2).FccItemName",
            ],
        )
        df4 = ld.get_history(
            universe = [ric],
            start = '2000-01-01',
            end = '2025-01-01',
            fields = [
                "TR.F.IncomeStatement(Period = FQ-3)",
                "TR.F.IncomeStatement(Period=FQ-3).FccItemName",
            ],
        )   
    except:
        continue
    # Concatenate the DataFrames for each RIC
    dfs = pd.concat([df1, df2, df3, df4])
    
    # Check for failed downloads
    if df1.empty:
        print(f"df1 empty for RIC {ric}.")
    elif df2.empty:
        print(f"df2 empty for RIC {ric}.")
    elif df3.empty:
        print(f"df3 empty for RIC {ric}.")
    elif df4.empty:
        print(f"df4 empty for RIC {ric}.")  
        continue
   
    dfs["RIC"] = ric 
    
    if dfs.empty:
        print(f"No data found for RIC {ric}.")
        empty_count += 1
        continue 
    
    # Append the concatenated DataFrame to the results list
    results.append(dfs)

# Concatenate all DataFrames in the results list into a single DataFrame
income_statements = pd.concat(results)

print(f"Number of RICs with no data: {empty_count}")

Processing RIC 1/41: LITE.OQ


c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `

Processing RIC 2/41: SAIA.OQ


c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `

Processing RIC 3/41: MRP.N
df1 empty for RIC MRP.N.
No data found for RIC MRP.N.
Processing RIC 4/41: CLH.N
Processing RIC 5/41: CMA.N
Processing RIC 6/41: AKAM.OQ
Processing RIC 7/41: VFC.N
Processing RIC 8/41: SAIC.OQ


...

Processing RIC 34/41: WMB.N
Processing RIC 35/41: YUM.N
Processing RIC 36/41: LIVN.OQ


c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Processing RIC 37/41: BFb.N
Processing RIC 38/41: WYNN.OQ
Processing RIC 39/41: WDC.OQ
Processing RIC 40/41: RYN.N
Processing RIC 41/41: CHX.OQ


c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
c:\Users\benny\OneDrive\Studium\Tübingen\DS_in_B&E\Masterarbeit\thesis_environment\Lib\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `

Number of RICs with no data: 2


In [28]:
# Work with a copy just to be safe
df = income_statements.copy()

# Reset index and rename to Date
df = df.reset_index().rename(columns={'index': 'Date'})

# Indicate statement type
df["Statement"] = "income_statement"

# Get rid of timestamp
df["Date"] = pd.to_datetime(df["Date"]).dt.date

# Save to CSV
df.to_csv("data/missing_income_statements.csv", index=False)

In [29]:
ld.close_session()